In [3]:
import eegPinelineDesign
import numpy as np
import random
import mne
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot') 
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import re
import json
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,FastICA
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from scipy.fftpack import fft,ifft
import math
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from scipy.signal import spectrogram,find_peaks_cwt,butter, lfilter
from mne.preprocessing.ica import ICA
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.cross_validation import train_test_split,ShuffleSplit
from sklearn.preprocessing import label_binarize,scale
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize,StandardScaler
eegPinelineDesign.change_file_directory('C:/Users/ning/Downloads/training set')
EDFfiles, Annotationfiles = eegPinelineDesign.split_type_of_files()

['64chlocs.elp', 'label_extraction.npy', 'label_extraction.pkl', 'pineline 13.csv', 'pipe line 10', 'pipe line 10.csv', 'pipeline 13.pickle', 's5d2_final.edf', 's5d2_final_annotations.txt', 's6n2_final.edf', 's6n2_final_annotations.txt', 'Sleep Stage Scoring Criteria.docx', 'spindle training.p', 'suj10_d1final.edf', 'suj10_d1final_annotations.txt', 'suj10_d2final.edf', 'suj10_d2final_annotations.txt', 'suj13_l2nap_day2 edited.edf', 'suj13_l2nap_day2 edited1.edf', 'suj13_l2nap_day2 edited1_annotations.txt', 'suj13_l2nap_day2 edited_annotations.txt', 'suj13_l2nap_day2 edited_C3.txt', 'suj13_l2nap_day2 edited_C4.txt', 'suj13_l2nap_day2 edited_F3.txt', 'suj13_l2nap_day2 edited_F4.txt', 'suj13_l2nap_day2 edited_O1.txt', 'suj13_l2nap_day2 edited_O2.txt', 'suj5_d1final.edf', 'suj5_d1final_annotations.txt', 'suj6_d1final.edf', 'suj6_d1final_annotations.txt', 'suj8_d1final.edf', 'suj8_d1final_annotations.txt', 'suj8_d2final.edf', 'suj8_d2final_annotations.txt', 'suj9_d1final.edf', 'suj9_d1final

In [4]:
#from eegPinelineDesign import CenterAtPeakOfWindow,Threshold_test,spindle_overlapping_test,used_windows_check,cut_segments
def Threshold_test(timePoint,raw,channelID,threshold = 0.3,steps=20):
    result = []
    for windowsize in np.linspace(0.1,5,steps):
        startPoint=timePoint-windowsize;endPoint=timePoint+windowsize
        start,stop=raw.time_as_index([startPoint,endPoint])
        tempSegment,timeSpan = raw[channelID,start:stop]
        filter_alpha=mne.filter.band_pass_filter(tempSegment[0,:],1000,8,12.5)
        RMS_alpha=np.sqrt(sum(filter_alpha**2)/len(filter_alpha))
        
        filter_spindle=mne.filter.band_pass_filter(tempSegment[0,:],1000,11,16.5)
        RMS_spindle=np.sqrt(sum(filter_spindle**2)/len(filter_spindle))
        
        filter_muscle=mne.filter.band_pass_filter(tempSegment[0,:],1000,30,40.5)
        RMS_muscle=np.sqrt(sum(filter_muscle**2)/len(filter_muscle))
    
        if (RMS_alpha/RMS_spindle <1.2) and (RMS_muscle < 5*10e-4):
            result.append(1)
        else:
            result.append(0)
    result = np.array(result)
    return sum(result[result==1])/len(result) >=threshold

def getOverlap(a,b):
    return max(0,min(a[1],b[1]) - max(a[0],b[0]))
def spindle_overlapping_test(spindles,timePoint,windowsize,tolerance=0.01):
    startPoint=timePoint-windowsize;endPoint=timePoint+windowsize
    return all(getOverlap([startPoint,endPoint],[instance-windowsize,instance+windowsize])<=tolerance for instance in spindles)

def used_windows_check(timePoint,used_time_windows,windowsize,tolerance=0.01):
    startPoint=timePoint-windowsize;endPoint=timePoint+windowsize
    return all(getOverlap([startPoint,endPoint],[lower,upper])<=tolerance for (lower,upper) in used_time_windows)

def cut_segments(raw,center,channelIndex,windowsize = 1.5):
    startPoint=center-windowsize;endPoint=center+windowsize
    start,stop=raw.time_as_index([startPoint,endPoint])
    tempSegment,timeSpan=raw[channelIndex,start:stop]
    return tempSegment,timeSpan

In [ ]:
spindle={};nonspindle={};
for num, EDFfileName in enumerate(EDFfiles):
    if EDFfileName == 'suj13_l2nap_day2 edited.edf' or EDFfileName =='suj13_l2nap_day2 edited1.edf':
        pass
    else:
        file_to_read,fileName=eegPinelineDesign.pick_sample_file(EDFfiles,n=num)
        channelList = ['F3','F4','C3','C4','O1','O2','ROC','LOC']
        raw = eegPinelineDesign.load_data(file_to_read,channelList,None, 100)#low pass data
        annotation_to_read = [x for x in Annotationfiles if fileName in x]
        file = pd.read_csv(annotation_to_read[0])
        labelFind = re.compile('spindle',eegPinelineDesign.re.IGNORECASE)
        spindles=[]# take existed annotations
        for row in file.iterrows():
            currentEvent = row[1][-1]
            if labelFind.search(currentEvent):
                spindles.append(row[1][0])# time of marker
        print('finish spindle annotation')
        
        channelList=['F3','F4','C3','C4','O1','O2']
        used_time_windows=[]
        spindle[fileName]={};nonspindle[fileName]={}
        for channelID, names in enumerate(channelList):
            spindle[fileName][names]=[];nonspindle[fileName][names]=[]
            for items in spindles:
                if Threshold_test(item,raw,channelID):
                    centered_marker=eegPinelineDesign.CenterAtPeakOfWindow(items,2,raw,channelID)
                    Segment,_ = cut_segments(raw,centered_marker,channelID)
                    spindle[fileName][names].append(Segment)
                    for max_iteration in range(10000):
                        startPoint=6;endPoint=raw.last_samp/1000-6
                        start,stop=raw.time_as_index([startPoint,endPoint])
                        S,T = raw[channelID,start:stop]
                        timePoint=np.random.choice(T,1)
                        if (Threshold_test(timePoint,raw,channelID)) and (spindle_overlapping_test(spindles,timePoint,1.5)) and (used_windows_check(timePoint,used_time_windows,1.5)):
                            Segment,_=cut_segments(raw,timePoint,channelID)
                            nonspindle[fileName][names].append(Segment)
                            used_time_windows.append([timePoint-1.5,timePoint+1.5)
                            break
                        else:
                            pass
                else:
                    pass

Extracting edf Parameters from s5d2_final.edf...
Setting channel info structure...
Creating Raw.info structure...
Reading 0 ... 3383999  =      0.000 ...  3383.999 secs...
Ready.
Fitting ICA to data using 8 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Using all PCA components: 8
    Searching for artifacts...
    found 2 artifacts by EOG 00
    found 1 artifact by skewness
    found 1 artifact by kurtosis
    found 1 artifact by variance
Artifact indices found:
    5, 6, 0, 0, 7
    Removing duplicate indices...
Ready.
Transforming to ICA space (8 components)
Zeroing out 4 ICA components
Low-pass filtering at 5e+02 Hz
Low-pass filtering at 1e+02 Hz
finish spindle annotation
Attenuation at stop frequency 7.5Hz is only 0.6dB.
Attenuation at stop frequency 10.5Hz is only 0.7dB.
Attenuation at stop frequency 29.5Hz is only 1.1dB.
Attenuation at stop frequency 13.0Hz is only 3.4dB.
Attenuation at stop frequency 10.5Hz is only 3.3dB.
Attenuat

In [ ]:
result={'spindle':temp_spindle,'non spindle':temp_nonspindle}
import pickle
with open('training data.p','wb') as handle:
    pickle.dump(result,handle)